### Processing LIAD-Framework ready data

In [1]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2
from joblib import Parallel, delayed
import math

### Train / Test data Processing

Train and Test data should contain:

1. "agent"        : Id of the agent

2. "started_at".  : Date and time. We assume the data you provide are UTC, then we convert to Asia/Tokyo.

3. "finished_at"  : Date and time. We assume the data you provide are UTC, then we convert to Asia/Tokyo.

4. "location_id"  : Id of the location

5. "latitude"       

6. "longitude"

7. "poi_category" : POI label of the location

Store the data as train.csv and test.csv in the processed folder

### Ground Truth Processing

For the evaluation purpose, you need to have a anomalous_agents.csv saved in processed folder,
the file ideally should contain a single column called 'agent', which contain the ids of all the anomalous agents in the simulation

In [ ]:
### Some of our data processing steps may or may not be important for you

train = '/Users/chanuka/Desktop/codespaces/neural_reeb/data/trail_4_sim1_fis/sim1_fis_sp_past_trajectories_with_POI.csv'
test = '/Users/chanuka/Desktop/codespaces/neural_reeb/data/trail_4_sim1_fis/sim1_fis_sp_future_trajectories_with_POI.csv'

train_data = pd.read_csv(train)
test_data = pd.read_csv(test)

eval_b_train = pd.read_parquet('/Users/chanuka/Desktop/codespaces/liad/data/files/evalb_stay_points_test_anomalous.parquet')
poi_data = pd.read_parquet('/Users/chanuka/Desktop/codespaces/liad/data/files/evalb_poi.parquet')
eval_b_train = eval_b_train.merge(poi_data, on='poi_id', how='left')
eval_b_train

eval_b_train.rename(columns={'agent_id': 'agent', 'start_datetime':'started_at', 'end_datetime':'finished_at', 'poi_id': 'location_id', 'category':'poi_category'}, inplace=True)
eval_b_train.drop(columns=['source', 'anomaly_type', 'anomaly'], inplace=True)

gt = pd.read_csv('/Users/chanuka/Desktop/codespaces/liad/data/trail_4_sim1_fis/anomalous_temporal_sim1.csv')
ground_truth = pd.read_csv('/Users/chanuka/Desktop/codespaces/liad/data/trail_4_sim1_fis/anomalous_agent_sim1.csv')

gt

eval_b_train = pd.read_parquet('/Users/chanuka/Desktop/codespaces/liad/data/files/evalb_stay_points_test_anomalous.parquet')
eval_b_train = eval_b_train[eval_b_train['anomaly'] == True]
eval_b_train.rename(columns={'agent_id': 'agent', 'start_datetime': 'anomaly_start_time', 'end_datetime': 'anomaly_end_time'}, inplace=True)
eval_b_train.to_csv('../processed/anomalous_temporal.csv', index= False)

### GT processing

In [ ]:
gt = pd.read_csv('/Users/chanuka/Desktop/codespaces/neural_reeb/data/trail_4_sim1_fis/anomalous_temporal_sim1.csv')
gt_new = pd.read_parquet('/Users/chanuka/Desktop/codespaces/liad/data/files/evalb_stay_points_test_anomalous.parquet')
gt_new['start_datetime'] = pd.to_datetime(gt_new['start_datetime'], utc=True).dt.tz_convert('Asia/Tokyo')
gt_new['end_datetime'] = pd.to_datetime(gt_new['end_datetime'], utc=True).dt.tz_convert('Asia/Tokyo')
gt_new.rename(columns={'agent_id': 'agent', 'start_datetime': 'started_at', 'end_datetime': 'finished_at'}, inplace=True) # should have agent, started_at, finished_at
gt_new = gt_new[gt_new['anomaly']== True]
gt_new = gt_new[['agent', 'started_at', 'finished_at']]

gt_new_agents = pd.DataFrame(gt_new.agent.unique(), columns=['agent'])

# format 1
gt_new_agents.to_csv('../processed/anomalous_agents.csv', index=False)

In [23]:
gt_new = split_by_time_bins(gt_new)

gt_new['time_segment'] = gt_new['started_at'].apply(assign_time_segment)
gt_new['day_of_week'] = gt_new['started_at'].dt.dayofweek
gt_new['day_type'] = gt_new['day_of_week'].apply(lambda x: 'weekend' if x >= 5 else 'weekday')

In [26]:
gt_new.to_csv('../processed/anomalous_segmented.csv', index=False)